Міністерство освіти і науки України 

Національний технічний університет України «КПІ ім. Ігоря Сікорського»

Факультет інформатики та обчислювальної техніки

Кафедра інформатики та програмної інженерії

Аналіз даних в інформаційно-управляючих системах

Виконанав студент:

ІП-01 Хернуф Валід Алі-Еддін

Київ 2022


---

# **Звіт до лабораторної роботи №8**

Імпортуємо всі потрібні бібліотеки та методи:

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
import collections
import pandas as p
import numpy as n
import nltk as nl
import pymorphy2
import re
import csv


Зчитаємо датасет з файлу "ukr_text.csv":

In [2]:
def read_dataset(path, separ, en):
    return p.read_csv(path, sep = separ, encoding = en)

dFrame = read_dataset("ukr_text.csv", ",", "utf-8")

виведимо інформацію про датасет:

In [3]:
dFrame.info()
dFrame.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1122 entries, 0 to 1121
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Id      1122 non-null   object
 1   Title   1122 non-null   object
 2   Body    1122 non-null   object
dtypes: object(3)
memory usage: 26.4+ KB


,Id,Title,Body
0,http://k.img.com.ua/rss/ua/4013798,Кличко покликав німецьких інвесторів до Києва,Київ - перспективний і відкритий ринок для біз...
1,http://k.img.com.ua/rss/ua/4001679,"З'явилося відео, як байкер почав стріляти у во...",З'явилося відео конфлікту між мотоциклістом...
2,http://k.img.com.ua/rss/ua/4001390,У центрі Києва посеред вулиці помер чоловік,У Києві на Бессарабській площі вранці в четвер...
3,http://k.img.com.ua/rss/ua/4001239,Нічний ураган перетворив Хрещатик на смітник,Київ вночі 16 серпня пережив найсильнішу грозу...
4,http://k.img.com.ua/rss/ua/4001227,Потоп у Києві: столицю накрив ураган з градом,Уночі Київ вкотре накрила негода. Найсильніший...


Видалимо непотрібний стовпчик:

In [4]:
def drop_columns(dFrame, columns):
    for column in columns:
        dFrame = dFrame.drop(column, axis = 1)
    return dFrame

dFrame = drop_columns(dFrame, ["Id"])

Поділяємо строки на токени, та видаляємо розмітки та тегів:

In [5]:
def to_tokens(dFrame):
    for i in range(dFrame.shape[0]):
        for j in range(dFrame.shape[1]):
            temp_text = dFrame.iloc[i, j]
            dFrame.iloc[i, j] = re.sub("[^А-Яа-яЁёЇїІіЄєҐґ'\-\s]", "", temp_text).split()

to_tokens(dFrame)
dFrame.info()
dFrame.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1122 entries, 0 to 1121
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Title   1122 non-null   object
 1   Body    1122 non-null   object
dtypes: object(2)
memory usage: 17.7+ KB


,Title,Body
0,"[Кличко, покликав, німецьких, інвесторів, до, ...","[Київ, -, перспективний, і, відкритий, ринок, ..."
1,"[З'явилося, відео, як, байкер, почав, стріляти...","[З'явилося, відео, конфлікту, між, мотоцикліст..."
2,"[У, центрі, Києва, посеред, вулиці, помер, чол...","[У, Києві, на, Бессарабській, площі, вранці, в..."
3,"[Нічний, ураган, перетворив, Хрещатик, на, смі...","[Київ, вночі, серпня, пережив, найсильнішу, гр..."
4,"[Потоп, у, Києві, столицю, накрив, ураган, з, ...","[Уночі, Київ, вкотре, накрила, негода, Найсиль..."


Завантажуємо дані з файлу "stop_words_ua.csv" та додаємо до стоп-слів ще деяки слова:

In [6]:
def csv_to_list(path, n_l, enc):
    with open(path, newline = n_l, encoding = enc) as s_w:
        return [element for sublist in list(csv.reader(s_w)) for element in sublist]

def get_stop_words(path, n_l, enc):
    stop_words = csv_to_list(path, n_l, enc)
    stop_words.remove("stopword")
    stop_words.extend(["тис", "грн", "вул", "cек", "хв", "обл", "кв", "пл", "напр", "гл", "о", "зам", "із"])
    return stop_words

stop_words = get_stop_words("stop_words_ua.csv", "", "utf-8")
print(stop_words)

['a', 'б', 'в', 'г', 'е', 'ж', 'з', 'м', 'т', 'у', 'я', 'є', 'і', 'аж', 'ви', 'де', 'до', 'за', 'зі', 'ми', 'на', 'не', 'ну', 'нх', 'ні', 'по', 'та', 'ти', 'то', 'ту', 'ті', 'це', 'цю', 'ця', 'ці', 'чи', 'ще', 'що', 'як', 'їй', 'їм', 'їх', 'її', 'або', 'але', 'ало', 'без', 'був', 'вам', 'вас', 'ваш', 'вже', 'все', 'всю', 'вся', 'від', 'він', 'два', 'дві', 'для', 'ким', 'мож', 'моя', 'моє', 'мої', 'міг', 'між', 'мій', 'над', 'нам', 'нас', 'наш', 'нею', 'неї', 'них', 'ніж', 'ній', 'ось', 'при', 'про', 'під', 'пір', 'раз', 'рік', 'сам', 'сих', 'сім', 'так', 'там', 'теж', 'тим', 'тих', 'той', 'тою', 'три', 'тут', 'хоч', 'хто', 'цей', 'цим', 'цих', 'час', 'щоб', 'яка', 'які', 'адже', 'буде', 'буду', 'будь', 'була', 'були', 'було', 'бути', 'вами', 'ваша', 'ваше', 'ваші', 'весь', 'вниз', 'вона', 'вони', 'воно', 'всею', 'всім', 'всіх', 'втім', 'геть', 'далі', 'двох', 'день', 'дуже', 'зате', 'його', 'йому', 'каже', 'кого', 'коли', 'кому', 'крім', 'куди', 'лише', 'люди', 'мало', 'мати', 'мене', 

Остаточно вирішуємо усі проблеми, які могли залишитися після поділу на токени:

In [7]:
def fix_porblems(dFrame, stop_words):
    for i in range(dFrame.shape[0]):
        for j in range(dFrame.shape[1]):
            for k in range(len(dFrame.iloc[i,j])):
                dFrame.iloc[i,j][k] = dFrame.iloc[i,j][k].lower()
            dFrame.iloc[i,j] = [x for x in dFrame.iloc[i,j] if (re.fullmatch("(\w+(\-|\'|\`)\w+|\w+)", x) and len(x) > 2)]
            dFrame.iloc[i,j] = [x for x in dFrame.iloc[i,j] if not (x in stop_words)]

fix_porblems(dFrame, stop_words)
dFrame.info()
dFrame.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1122 entries, 0 to 1121
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Title   1122 non-null   object
 1   Body    1122 non-null   object
dtypes: object(2)
memory usage: 17.7+ KB


,Title,Body
0,"[кличко, покликав, німецьких, інвесторів, києва]","[київ, перспективний, відкритий, ринок, бізнес..."
1,"[з'явилося, відео, байкер, почав, стріляти, во...","[з'явилося, відео, конфлікту, мотоциклістом, в..."
2,"[центрі, києва, вулиці, помер, чоловік]","[києві, бессарабській, площі, вранці, четвер, ..."
3,"[нічний, ураган, перетворив, хрещатик, смітник]","[київ, вночі, серпня, пережив, найсильнішу, гр..."
4,"[потоп, києві, столицю, накрив, ураган, градом]","[уночі, київ, вкотре, накрила, негода, найсиль..."


Лемматизуємо слова:

In [8]:
def lematize(dFrame):
    uk_l = pymorphy2.MorphAnalyzer(lang='uk')
    for i in range(dFrame.shape[0]):
        for j in range(dFrame.shape[1]):
            for k in range(len(dFrame.iloc[i,j])):
                dFrame.iloc[i,j][k] = uk_l.parse(dFrame.iloc[i,j][k])[0].normal_form

lematize(dFrame)
dFrame.info()
dFrame.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1122 entries, 0 to 1121
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Title   1122 non-null   object
 1   Body    1122 non-null   object
dtypes: object(2)
memory usage: 17.7+ KB


,Title,Body
0,"[кличко, покликати, німецький, інвестор, київ]","[кий, перспективний, відкритий, ринок, бізнес,..."
1,"[з'явитися, відео, байкер, почати, стріляти, в...","[з'явитися, відео, конфлікт, мотоцикліст, воді..."
2,"[центр, київ, вулиця, померти, чоловік]","[кий, бессарабський, площа, вранці, четвер, се..."
3,"[нічний, ураган, перетворити, хрещатик, смітник]","[кий, вночі, серпень, пережити, найсильніший, ..."
4,"[потоп, кий, столиця, накрити, ураган, град]","[уночі, кий, вкотре, накрити, негода, найсильн..."


Знаходимо 10 найпопуляірнійших слів в усіх "Body":

In [10]:
def get_top_words(dFrame):
    bodies = []
    for i in range(dFrame.shape[0]):
        bodies += dFrame.iloc[i]["Body"]
    return dict(collections.Counter(bodies).most_common(10))

top_words_dict = get_top_words(dFrame)
print(top_words_dict)
top_words = list(top_words_dict.keys())

{'україна': 1195, 'новий': 589, 'компанія': 574, 'корреспондент': 571, 'країна': 455, 'світ': 449, 'український': 444, 'повідомлятися': 433, 'перший': 426, 'один': 394}


Рахуємо та виводимо для них метрики tf-idf:

In [11]:
def identity_tokenizer(text):
    return text

def df_tfidf(dFrame, topWords):
    tokenized_list = []
    for i in range(dFrame.shape[0]):
        tokenized_list.append(dFrame.iloc[i]["Body"])
    tfidf = TfidfVectorizer(tokenizer = identity_tokenizer, lowercase = False)
    vectors = tfidf.fit_transform(tokenized_list)
    feature_names = tfidf.get_feature_names_out()
    denselist = vectors.todense().tolist()
    d_l, f_n = top_n_tfidf(feature_names, denselist, topWords)
    return p.DataFrame(n.array(d_l).T.tolist(), columns = f_n)

def top_n_tfidf(feature_names, denselist, topWords):
    f_n = []
    d_l = []
    feature_names = n.array(feature_names)
    denselist = n.array(denselist)
    for i in range(len(feature_names)):
        is_there = False
        for e2 in topWords:
            if feature_names[i] == e2:
                is_there = True
        if is_there:
            f_n.append(feature_names[i])
            temp = []
            for j in range(len(denselist[:,i])):
                temp.append(denselist[j][i])
            d_l.append(temp)
    return d_l, f_n

df = df_tfidf(dFrame, top_words)
df

,компанія,корреспондент,країна,новий,один,перший,повідомлятися,світ,україна,український
0,0.074764,0.000000,0.039572,0.017326,0.054679,0.018513,0.015480,0.000000,0.059960,0.075369
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.068100,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.065389,0.000000,0.055535,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.036248,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
1117,0.093566,0.032873,0.015238,0.013344,0.021056,0.007129,0.000000,0.021802,0.040406,0.050790
1118,0.013783,0.036723,0.014591,0.000000,0.013441,0.034131,0.000000,0.000000,0.071853,0.006948
1119,0.053115,0.040432,0.033736,0.039389,0.041436,0.000000,0.000000,0.000000,0.017039,0.010709
1120,0.043648,0.022151,0.184820,0.000000,0.028376,0.028822,0.000000,0.088146,0.151691,0.029334


Створюємо та виводимо "Bag of Word" для всіх нормалізованих слів:

In [12]:
def calculateBOW(wordset, l_doc):
    tf_diz = dict.fromkeys(wordset, 0)
    for word in l_doc:
        tf_diz[word] = l_doc.count(word)
    return tf_diz

def get_wordset(dFrame):
    wordset = None
    for i in range(dFrame.shape[0]):
        if (i == 1):
            wordset = n.union1d(dFrame.iloc[0]["Body"], dFrame.iloc[1]["Body"])
        elif (i > 1):
            wordset = n.union1d(wordset, dFrame.iloc[i]["Body"])
    return wordset

def dataFrame_bowl(dFrame):
    bow = []
    wordset = get_wordset(dFrame)
    for i in range(dFrame.shape[0]):
        bow.append(calculateBOW(wordset, dFrame.iloc[i]["Body"]))
    return p.DataFrame(bow)

df_bow = dataFrame_bowl(dFrame)
df_bow.head(10)

,а-ля,аахен,аахенський,абаджян,абатство,абдель,абдул-джаббар,абдулла,абдумалик,абер,...,ґас,ґедун,ґодар,ґрунт,ґрунтовий,ґрунтовний,ґрунтовно,ґрунтуватися,ґудзик,ґути
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Цей самий "Bag of Word" на прикладі 10 найчастіших слів:

In [13]:
df_useful = df_bow[top_words]
df_useful.head(15)

,україна,новий,компанія,корреспондент,країна,світ,український,повідомлятися,перший,один
0,4,1,4,0,2,0,4,1,1,3
1,2,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,1
3,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0
5,1,0,1,1,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,1,1,0
8,0,0,0,1,0,0,0,2,0,0
9,0,0,0,1,0,0,1,3,1,0
